In [27]:
pip install -r requirements

ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/16/2e/86f24451c2d530c88daf997cb8d6ac622c1d40d19f5a031ed68a4b73a374/numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/87/03/fe50521919aa981f6a1c197037da4623a267b0e5f42246d69ba048e86da3/pandas-2.2.0-cp312-cp312-win_amd64.whl.metadata
  Using cached pandas-2.2.0-cp312-cp312-win_amd64.whl.metadata (19 kB)


In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import gensim
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
from sklearn.metrics import confusion_matrix
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv("Twitter_Data.csv")

df.head()

In [ ]:
df = df.dropna()
df.isna().sum()

In [ ]:
df.count()

In [ ]:
df["category"].value_counts()

In [ ]:
bow_vectorizer = CountVectorizer( stop_words='english')  #max_df=0.90, min_df=2, max_features=1000,
bow_lg = bow_vectorizer.fit_transform(df["clean_text"])
bow_lg.shape

In [ ]:
tfidf_vectorizer = TfidfVectorizer( stop_words='english') #max_df=0.90, min_df=2, max_features=1000,
tfidf_lg = tfidf_vectorizer.fit_transform(df["clean_text"])
tfidf_lg.shape

In [ ]:
df["category"] = df["category"].replace(-1, 2)

df["category"].unique()

In [ ]:

xtrain_bow, xvalid_bow, ytrain_bow, yvalid_bow = train_test_split(bow_lg, df['category'] , random_state=42, test_size=0.3)


lreg = LogisticRegression() 
# # training the model
lreg.fit(xtrain_bow, ytrain_bow) 

In [ ]:
test_pred = lreg.predict_proba(xvalid_bow)
test_pred_int = test_pred[:,1] >= 0.3 
test_pred_int = test_pred_int.astype(np.int64) 

pred_bow_lg = lreg.predict(xvalid_bow)

# test['label'] = test_pred_int 
# submission = test[['id','label']] 
# submission.to_csv('sub_lreg_bow.csv', index=False) # writing data to a CSV file




In [ ]:


warnings.filterwarnings("ignore", category=FutureWarning)
# Accuracy
accuracy_bow_lg = accuracy_score(yvalid_bow, pred_bow_lg)
print(f'Accuracy: {accuracy_bow_lg:.2f}')

# Precision
precision_bow_lg = precision_score(yvalid_bow, pred_bow_lg ,  average="macro")
print(f'Precision: {precision_bow_lg:.2f}')

# Recall
recall_bow_lg = recall_score(yvalid_bow, pred_bow_lg  ,  average="macro" )
print(f'Recall: {recall_bow_lg:.2f}')

# F1 Score
f1_bow_lg = f1_score(yvalid_bow, pred_bow_lg   ,  average="macro")
print(f'F1 Score: {f1_bow_lg:.2f}')


In [ ]:
values = [accuracy_bow_lg, precision_bow_lg, recall_bow_lg, f1_bow_lg]
labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

plt.bar(labels, values, color=['blue', 'green', 'orange', 'red'])
plt.ylim(0, 1)  # Set y-axis limit to 0-1 for better visualization
plt.title('Performance Metrics')
plt.ylabel('Score')
plt.show()



In [ ]:


cm = confusion_matrix(yvalid_bow, pred_bow_lg)

# Plot the confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# train_tfidf = tfidf_lg[:132969,:] 
# test_tfidf = tfidf[132970:,:] 


# train_label1 = df['category'][:132969]
# test_label1 = df['category'][132970:]

xtrain_tfidf, xvalid_tfidf, ytrain_tfidf, yvalid_tfidf = train_test_split(tfidf_lg, df['category'] , random_state=42, test_size=0.3)


lreg = LogisticRegression() 
# # training the model
lreg.fit(xtrain_bow, ytrain_bow) 

lreg.fit(xtrain_tfidf, ytrain_tfidf) 
prediction = lreg.predict_proba(xvalid_tfidf) 
prediction_int = prediction[:,1] >= 0.3
prediction_int = prediction_int.astype(np.int64) 

pred_tf_lg = lreg.predict(xvalid_tfidf)


In [ ]:
# Accuracy


warnings.filterwarnings("ignore", category=FutureWarning)

accuracy_tf_lg = accuracy_score(yvalid_tfidf, pred_tf_lg)
print(f'Accuracy: {accuracy_tf_lg:.2f}')

# Precision
precision_tf_lg = precision_score(yvalid_tfidf, pred_tf_lg ,  average="macro")
print(f'Precision: {precision_tf_lg:.2f}')

# Recall
recall_tf_lg = recall_score(yvalid_tfidf, pred_tf_lg  ,  average="macro" )
print(f'Recall: {recall_tf_lg:.2f}')

# F1 Score
f1_tf_lg = f1_score(yvalid_tfidf, pred_tf_lg   ,  average="macro")
print(f'F1 Score: {f1_tf_lg:.2f}')

In [ ]:
values = [accuracy_tf_lg, precision_tf_lg, recall_tf_lg, f1_tf_lg]
labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

plt.bar(labels, values, color=['blue', 'green', 'orange', 'red'])
plt.ylim(0, 1)  # Set y-axis limit to 0-1 for better visualization
plt.title('Performance Metrics')
plt.ylabel('Score')
plt.show()


In [ ]:


cm = confusion_matrix(yvalid_tfidf, pred_tf_lg)

# Plot the confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

## XGBOOST

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_model1 = XGBClassifier().fit(xtrain_bow, ytrain_bow) 
prediction1 = xgb_model1.predict(xvalid_bow)

In [ ]:


warnings.filterwarnings("ignore", category=FutureWarning)

accuracy_bow_xg = accuracy_score(yvalid_bow, prediction1)
print(f'Accuracy: {accuracy_bow_xg:.2f}')

# Precision
precision_bow_xg = precision_score(yvalid_bow, prediction1 ,  average="macro")
print(f'Precision: {precision_bow_xg:.2f}')

# Recall
recall_bow_xg = recall_score(yvalid_bow, prediction1  ,  average="macro" )
print(f'Recall: {recall_bow_xg:.2f}')

# F1 Score
f1_bow_xg = f1_score(yvalid_bow, prediction1   ,  average="macro")
print(f'F1 Score: {f1_bow_xg:.2f}')

In [ ]:
values = [accuracy_bow_xg, precision_bow_xg, recall_bow_xg, f1_bow_xg]
labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

plt.bar(labels, values, color=['blue', 'green', 'orange', 'red'])
plt.ylim(0, 1)  # Set y-axis limit to 0-1 for better visualization
plt.title('Performance Metrics')
plt.ylabel('Score')
plt.show()

In [ ]:


cm1 = confusion_matrix(yvalid_bow, prediction1)

# Plot the confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm1 , annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:

xgb_model = XGBClassifier().fit(xtrain_tfidf, ytrain_tfidf) 
prediction2 = xgb_model.predict(xvalid_tfidf) 

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

accuracy_tf_xg = accuracy_score(yvalid_tfidf, prediction2)
print(f'Accuracy: {accuracy_tf_xg:.2f}')

# Precision
precision_tf_xg  = precision_score(yvalid_tfidf, prediction2 ,  average="macro")
print(f'Precision: {precision_tf_xg :.2f}')

# Recall
recall_tf_xg  = recall_score(yvalid_tfidf, prediction2  ,  average="macro" )
print(f'Recall: {recall_tf_xg :.2f}')

# F1 Score
f1_tf_xg  = f1_score(yvalid_tfidf, prediction2   ,  average="macro")
print(f'F1 Score: {f1_tf_xg :.2f}')

In [ ]:
values = [accuracy_tf_xg , precision_tf_xg , recall_tf_xg , f1_tf_xg ]
labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

plt.bar(labels, values, color=['blue', 'green', 'orange', 'red'])
plt.ylim(0, 1)  # Set y-axis limit to 0-1 for better visualization
plt.title('Performance Metrics')
plt.ylabel('Score')
plt.show()

In [ ]:


cm2 = confusion_matrix(yvalid_tfidf, prediction2)

# Plot the confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm2, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
testr4 = bow_vectorizer.transform(["I am ok with the recent policies implemented by the government. #PoliticalFrustration"])

lreg.predict(testr4)